You don't have to read the code, just run it and be impressed by the beautiful dashboard (or file an issue if the dashboard does not look good on your computer, stating the exact version of your operating system.)

In [1]:
import mkl
mkl.set_num_threads(1)

from skillmodels import SkillModel
from skillmodels.estimation.likelihood_function import log_likelihood_per_individual
from estimagic.optimization.optimize import maximize

import json
import numpy as np
import pandas as pd
from functools import partial

In [2]:
with open('model_short.json', 'r') as f:
    model = json.load(f)
    
data = pd.read_stata('chs_test_ex2.dta')
for var in ['caseid', 'period']:
    data[var] = data[var].astype(int)
data = data.query('period <= 4')
data.set_index(['caseid', 'period'], inplace=True)
data.head()

y1        y2        y3        y4        y5        y6  \
caseid period                                                               
1      1       1.909221  2.053261  1.679474  1.205891  2.195575  1.499965   
       2       0.925990  1.828494  1.412966  1.235554  0.636243  1.534268   
       3       1.957160  2.251265 -0.373637  2.875391  1.838843  2.253621   
       4       1.236615  1.160494  3.001797  1.181049  2.170630  1.125383   
2      1       2.951471  1.704928  1.650190  1.329379  0.294313  1.510699   

                     y7        y8        y9        Q1  ...  dy3  dy4  dy5  \
caseid period                                          ...                  
1      1       0.873044  1.790903  1.191478  1.299120  ...  1.0  1.0  1.0   
       2       0.873044  1.790903  1.191478  1.299120  ...  1.0  1.0  1.0   
       3       0.873044  1.790903  1.191478  1.299120  ...  1.0  1.0  1.0   
       4       0.873044  1.790903  1.191478  1.299120  ...  1.0  1.0  1.0   
2      1       2.009538  2.387094  1.403985  1.914992  ...  1.0  1.0  1.0   

               dy6  dy7  dy8  dy9  dQ1        x1   x2  
caseid period                                          
1      1       1.0  1.0  1.0  1.0  1.0  0.473032  1.0  
       2       1.0  1.0  1.0  1.0  1.0  0.473032  1.0  
       3       1.0  1.0  1.0  1.0  1.0  0.473032  1.0  
       4       1.0  1.0  1.0  1.0  1.0  0.473032  1.0  
2      1       1.0  1.0  1.0  1.0  1.0  0.493638  1.0  

[5 rows x 22 columns]

In [3]:
def estimate_skillmodel(model_dict, dataset, estimator):
    sm = SkillModel(model_dict=model_dict, dataset=dataset, estimator=estimator)
    sm_log_like = _skillmodel_criterion(sm=sm)
    params_df = _skillmodel_params(
        bounds=sm.bounds_list(), 
        start_params=sm.start_params,
        param_names=sm.param_names('short'))

    res = maximize(
        criterion=sm_log_like,
        params=params_df,
        algorithm="scipy_L-BFGS-B",
        constraints=None,
        db_options={'rollover': 500}
    )

    return res
    
def _skillmodel_criterion(sm):
    kwargs = sm.likelihood_arguments_dict(params_type="short")
    def log_likelihood(params):
        arr_params = params.values
        return log_likelihood_per_individual(arr_params, **kwargs).sum()
    return log_likelihood


def _skillmodel_params(bounds, start_params, param_names):
    params_df = pd.DataFrame(np.array(bounds), columns=['lower', 'upper'])
    params_df.replace({
        'lower': {None: - np.inf}, 
        'upper': {None: np.inf}},
        inplace=True)
    params_df['value'] = start_params
    params_df['fixed'] = False
    name_tuples = [x.split('__', 2) for x in param_names]
    multi_ind = pd.MultiIndex.from_tuples(name_tuples)
    params_df.set_index(multi_ind, inplace=True)
    category = params_df.index.get_level_values(0)
    period = params_df.index.get_level_values(1)
    params_df['group'] = category + '__' + period
    
    replace_dict = {
        'gamma__0': 'trans_coeff', 'phi__0': 'trans_coeff',
        'lincoeff__0': 'trans_coeff'}
    params_df['group'].replace(replace_dict, inplace=True)
    params_df.loc['delta', 'group'] = None
    params_df.loc['R', 'group'] = None
    return params_df
    

In [ ]:
sm = SkillModel(model_dict=model, dataset=data, estimator='chs')
sm_log_like = _skillmodel_criterion(sm=sm)
params_df = _skillmodel_params(
    bounds=sm.bounds_list(), 
    start_params=sm.start_params,
    param_names=sm.param_names('short'))
params_df

/home/janos/anaconda3/envs/estimagic_presentation/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2874: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


lower  upper  value  fixed               group
delta           0 y1__x1         -inf    inf   1.00  False                None
                  y2__x1         -inf    inf   1.00  False                None
                  y3__x1         -inf    inf   1.00  False                None
                  y4__x1         -inf    inf   1.00  False                None
                  y5__x1         -inf    inf   1.00  False                None
                  y6__x1         -inf    inf   1.00  False                None
                  y7__x1         -inf    inf   1.00  False                None
                  y8__x1         -inf    inf   1.00  False                None
                  y9__x1         -inf    inf   1.00  False                None
                1 y1__x1         -inf    inf   1.00  False                None
                  y2__x1         -inf    inf   1.00  False                None
                  y3__x1         -inf    inf   1.00  False                None
                  y4__x1         -inf    inf   1.00  False                None
                  y5__x1         -inf    inf   1.00  False                None
                  y6__x1         -inf    inf   1.00  False                None
                2 y1__x1         -inf    inf   1.00  False                None
                  y2__x1         -inf    inf   1.00  False                None
                  y3__x1         -inf    inf   1.00  False                None
                  y4__x1         -inf    inf   1.00  False                None
                  y5__x1         -inf    inf   1.00  False                None
                  y6__x1         -inf    inf   1.00  False                None
                3 y1__x1         -inf    inf   1.00  False                None
                  y2__x1         -inf    inf   1.00  False                None
                  y3__x1         -inf    inf   1.00  False                None
                  y4__x1         -inf    inf   1.00  False                None
                  y5__x1         -inf    inf   1.00  False                None
                  y6__x1         -inf    inf   1.00  False                None
                  Q1__constant   -inf    inf   1.00  False                None
                  Q1__x1         -inf    inf   1.00  False                None
H               0 fac1__y2       -inf    inf   1.00  False                H__0
...                               ...    ...    ...    ...                 ...
R               1 y2              0.0    inf   0.50  False                None
                  y3              0.0    inf   0.50  False                None
                  y4              0.0    inf   0.50  False                None
                  y5              0.0    inf   0.50  False                None
                  y6              0.0    inf   0.50  False                None
                2 y1              0.0    inf   0.50  False                None
                  y2              0.0    inf   0.50  False                None
                  y3              0.0    inf   0.50  False                None
                  y4              0.0    inf   0.50  False                None
                  y5              0.0    inf   0.50  False                None
                  y6              0.0    inf   0.50  False                None
                3 y1              0.0    inf   0.50  False                None
                  y2              0.0    inf   0.50  False                None
                  y3              0.0    inf   0.50  False                None
                  y4              0.0    inf   0.50  False                None
                  y5              0.0    inf   0.50  False                None
                  y6              0.0    inf   0.50  False                None
                  Q1              0.0    inf   0.50  False                None
Q               0 fac1            0.0    inf   0.50  False                Q__0
   

In [ ]:
res = estimate_skillmodel(model_dict=model, dataset=data, estimator='chs')

/home/janos/anaconda3/envs/estimagic_presentation/lib/python3.7/site-packages/IPython/core/async_helpers.py:67: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


Bokeh app running at: http://localhost:5039/
